In [2]:
from scipy.stats import mannwhitneyu
import itertools
import numpy as np
from collections import defaultdict
from typing import Callable, Optional
import csv
import pandas as pd
import os
import utils

import utils


def is_valid_data_file(file_name:str) -> bool:
    return file_name.endswith("json") or file_name.endswith("txt")


def get_mean_for_combinations(df: pd.DataFrame, 
                       independent_variables: list[str], 
                       dependent_variables: list[str]) -> pd.DataFrame:

    # ensure all the columns are present in the df
    for col in independent_variables+dependent_variables:
        if col not in df:
            raise Exception(f"The column {col} is not in the dataframe\n\t(columns are {list(df.columns)})")
    assert(all(col in df for col in independent_variables))
    assert(dependent_variable in df for dependent_variable in dependent_variables)
    
    grouped = df.groupby(independent_variables, dropna=False)[dependent_variables].mean().reset_index()
    
    return grouped


import json
import os


def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        problem_name = item["problem_name"]
        pRef_method = item["pRef_method"]
        pRef_size = item["sample_size"]

        entries = item["results_by_tree"]

        def get_modified_entry(entry):
            entry["problem"] = problem_name
            entry["pRef_method"] = pRef_method
            entry["pRef_size"] = pRef_size

            errors = entry["results"]
            entry = entry | errors
            del entry["results"]

            if "order_tree" in entry:
                del entry["order_tree"]

            return entry

        entries = list(map(get_modified_entry, entries))
        return entries

    return [entry for item in data for entry in item_to_list_of_entries(item)]

def convert_accuracy_data_to_df(input_directory, output_filename):

    all_dicts = []
    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        # Construct full file path
        file_path = os.path.join(input_directory, filename)

        # Check if the file is a JSON file
        if not os.path.isfile(file_path):
            continue

        if not is_valid_data_file(file_path):
            continue

        with open(file_path, 'r') as file:
            data = json.load(file)
            entries = json_to_entries(data)
            all_dicts.extend(entries)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_dicts)

    # Write the DataFrame to a CSV file
    df.to_csv(output_filename, index=False)
    
    

def json_to_tree_data(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        surrounding_information = {prop: item[prop]
                                   for prop in ["problem_name", "pRef_method"]}
        surrounding_information = {"problem": item["problem_name"],
                                   "pRef_method": item["pRef_method"]}

        entries = item["results_by_tree"]
        entries = [thing for thing in entries if "order_tree" in thing]  

        def convert_order_tree(order_tree, accumulator = None, current_depth: int = 0):
            if accumulator is None:
                accumulator = defaultdict(list)
            accumulator[current_depth].append(order_tree["own"])
            if len(order_tree["matching"]) > 0:
                convert_order_tree(order_tree["matching"], accumulator, current_depth+1)

            if len(order_tree["unmatching"]) > 0:
                convert_order_tree(order_tree["unmatching"], accumulator, current_depth+1)

            return accumulator
        def convert_tree_to_averages_by_level(entry):
            ps_search_info = {prop: entry[prop]
                                   for prop in ["ps_budget", "ps_population", "metrics"]}
            tree_structure = entry["order_tree"]
            just_depths = convert_order_tree(tree_structure)
            #average_orders_by_depth = {f"average_at_{depth}": np.average(orders)
            #                  for depth, orders in just_depths.items()}
            #standard_deviations = {f"sd_at_{depth}": np.std(orders)
            #                  for depth, orders in just_depths.items()}
            #overall_average = {"overall_average": np.average(list(itertools.chain(*(just_depths.values()))))}
            core_info_trees = [{"depth": depth,
                               "order": order}
                               for depth in just_depths
                                for order in just_depths[depth]
                               ]
            core_info_trees = [surrounding_information | ps_search_info | core_tree
            for core_tree in core_info_trees]
            return core_info_trees


        entries = list(map(convert_tree_to_averages_by_level, entries))
        return entries

    return [entry for item in data for entry in item_to_list_of_entries(item)]

def convert_tree_data_to_df(input_directory, output_filename):

    all_dicts = []
    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        # Construct full file path
        file_path = os.path.join(input_directory, filename)

        # Check if the file is a JSON file
        if not os.path.isfile(file_path):
            continue

        if not is_valid_data_file(file_path):
            continue


        with open(file_path, 'r') as file:
            data = json.load(file)
            entries = json_to_tree_data(data)
            all_dicts.extend(entries)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_dicts)

    # Write the DataFrame to a CSV file
    df.to_csv(output_filename, index=False)
    
    
def filter_dataframe(df, **kwargs):
    df = df.copy()  # Make a copy of the DataFrame to avoid modifying the original
    for col, value in kwargs.items():
        if col in df.columns:
            df = df[df[col] == value]
        else:
            raise ValueError(f"Column '{col}' not found in dataframe.")
    return df
        

    
    

def prettify_kind_column(df):
    kind_dict = {"variance":"PS-W",
                 "variance estimated_atomicity": "PS-WA",
                 "simplicity variance": "PS-SW",
                 "simplicity variance estimated_atomicity" :"PS-SWA"}
    
    df['kind'] = df.apply(
    lambda row: (
        kind_dict[row['metrics']] if row['kind'] == 'ps' else
        'Trad.' if row['kind'] == 'naive' else
        'IAI' if row['kind'] == 'iai' else
        row['kind']
    ),
    axis=1
)
    
    

    
    

In [3]:
#run_location = r"/Users/gian/Desktop/CondorResults/VDT/compareown/run3/"
run_location = r"C:\Users\gac8\Desktop\CondorResults\VDT\compareown\all_final_runs"

results_csv = os.path.join(run_location, "results.csv")
tree_data_csv = os.path.join(run_location, "tree_data.csv")


convert_accuracy_data_to_df(os.path.join(run_location, "data"), results_csv)
convert_tree_data_to_df(os.path.join(run_location, "data"), tree_data_csv)


In [4]:

accuracy_data = pd.read_csv(results_csv)
prettify_kind_column(accuracy_data)
#tree_data = pd.read_csv(tree_data_csv)

display(accuracy_data)
#display(accuracy_data.dtypes)
#display(tree_data)


#prettify_kind_column(tree_data)


for kind in accuracy_data["kind"].unique():
    matching_rows = accuracy_data[accuracy_data['kind'] == kind]
    print(f"For the tree kind {kind}, there are {matching_rows.shape[0]} rows")

#headers = "kind,depth,ps_budget,ps_population,avoid_ancestors,metrics,problem,pRef_method,mse,mae,r_sq,evs"

,kind,depth,problem,pRef_method,pRef_size,mse,mae,r_sq,evs,ps_budget,ps_population,avoid_ancestors,metrics,cp
0,Trad.,2,SAT_S,uniform,10000,10.428119,2.562856,0.244222,0.244534,NaN,NaN,NaN,NaN,NaN
1,Trad.,3,SAT_S,uniform,10000,8.912910,2.392481,0.354036,0.354210,NaN,NaN,NaN,NaN,NaN
2,Trad.,4,SAT_S,uniform,10000,7.868440,2.238714,0.429734,0.430817,NaN,NaN,NaN,NaN,NaN
3,Trad.,5,SAT_S,uniform,10000,6.944226,2.096012,0.496717,0.497513,NaN,NaN,NaN,NaN,NaN
4,Trad.,6,SAT_S,uniform,10000,6.355751,2.011702,0.539367,0.539568,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113515,IAI,2,BT,Tabu,10000,29.001422,4.231988,0.134516,0.134791,NaN,NaN,NaN,NaN,0.25
113516,IAI,3,BT,Tabu,10000,25.743493,3.994729,0.231742,0.231858,NaN,NaN,NaN,NaN,0.25
113517,IAI,4,BT,Tabu,10000,23.159979,3.810264,0.308841,0.308907,NaN,NaN,NaN,NaN,0.25
113518,IAI,5,BT,Tabu,10000,20.122597,3.606257,0.399485,0.399681,NaN,NaN,NaN,NaN,0.25


For the tree kind Trad., there are 26880 rows
For the tree kind PS-W, there are 26880 rows
For the tree kind PS-WA, there are 26880 rows
For the tree kind PS-SW, there are 9600 rows
For the tree kind PS-SWA, there are 9600 rows
For the tree kind IAI, there are 13680 rows


In [5]:
    
    
def generate_statistical_test_data(accuracy_data: pd.DataFrame, input_directory, output_filename):
    depths = [3, 4, 5]
    usable_data = filter_dataframe(accuracy_data, pRef_size = 10000)
    usable_data = usable_data[usable_data["depth"].isin(depths)]
    
    result_column = "r_sq"
    
    def winning_competitor_for_competition_and_values(problem: str, depth: int, metaheuristic: str) -> (str, np.ndarray):
        for_each_method = {tree_method: filter_dataframe(usable_data, problem = problem, depth = depth, pRef_method = metaheuristic, kind = tree_method)[result_column]
                           for tree_method in {"PS-SW", "PS-SWA", "IAI", "Trad."}}
        
        iai_average = np.average(for_each_method["IAI"])
        naive_average = np.average(for_each_method["Trad."])
        
        
        winning_competitor_method = "IAI" if iai_average > naive_average else "Trad."
        
        p_value_between_w_and_competitor = mannwhitneyu(for_each_method["PS-SW"], for_each_method[winning_competitor_method], alternative="greater").pvalue
        p_value_between_wa_and_competitor = mannwhitneyu(for_each_method["PS-SWA"], for_each_method[winning_competitor_method], alternative="greater").pvalue
        
        return {"problem": problem,
                "depth": depth,
                "metaheuristic": metaheuristic,
                "p_value_sw":p_value_between_w_and_competitor,
                "p_value_swa": p_value_between_wa_and_competitor,
                "winning_competitor": winning_competitor_method}
    
    
    
    all_problems = usable_data["problem"].unique()
    all_metaheuristics = usable_data["pRef_method"].unique()
    
    dicts = [winning_competitor_for_competition_and_values(problem=problem, depth = depth, metaheuristic=metaheuristic)
             for problem in all_problems
             for depth in depths
             for metaheuristic in all_metaheuristics]
    
    return pd.DataFrame(dicts)
        
        
    

statistical_data = generate_statistical_test_data(accuracy_data, None, None)

pivot_table = statistical_data.pivot_table(index=["problem", "depth", "metaheuristic"], 
                                            values =["p_value_sw", "p_value_swa"])
display(statistical_data)
display(pivot_table)

,problem,depth,metaheuristic,p_value_sw,p_value_swa,winning_competitor
0,SAT_S,3,uniform,1.000000e+00,1.000000e+00,Trad.
1,SAT_S,3,GA,2.440937e-03,1.000000e+00,Trad.
2,SAT_S,3,SA,9.999974e-01,1.000000e+00,Trad.
3,SAT_S,3,Tabu,1.001752e-04,1.000000e+00,Trad.
4,SAT_S,4,uniform,1.000000e+00,1.000000e+00,Trad.
...,...,...,...,...,...,...
67,BT,4,Tabu,9.963082e-01,1.000000e+00,Trad.
68,BT,5,uniform,1.000000e+00,1.000000e+00,Trad.
69,BT,5,GA,4.464741e-31,2.666489e-11,Trad.
70,BT,5,SA,1.000000e+00,1.000000e+00,Trad.


p_value_sw   p_value_swa
problem depth metaheuristic                            
BT      3     GA             3.457953e-34  6.250812e-14
              SA             4.816641e-01  1.000000e+00
              Tabu           9.984247e-01  1.000000e+00
              uniform        1.000000e+00  1.000000e+00
        4     GA             6.475038e-33  3.506722e-12
...                                   ...           ...
SAT_S   4     uniform        1.000000e+00  1.000000e+00
        5     GA             2.302273e-02  1.000000e+00
              SA             1.000000e+00  1.000000e+00
              Tabu           4.593448e-03  1.000000e+00
              uniform        1.000000e+00  1.000000e+00

[72 rows x 2 columns]

In [6]:
def bold_max(row):
    row_as_numbers = [float(item[:-1]) for item in row]
    max_number = max(row_as_numbers)
    
    return ['font-weight: bold' if item == max_number else '' for item in row_as_numbers]

    
def style_pivot_table(pivot_table):
    custom_column_order = ['PS-SW', 'PS-SWA', 'IAI', 'Trad.']

    pivot_table = pivot_table.mul(100).round(1).astype(str) + "%"    

    # Reorder columns based on custom order
    pivot_table = pivot_table.reindex(columns=custom_column_order)

    styled_df = pivot_table.style.apply(bold_max, axis=1)

    return styled_df

def put_latex_tables_side_by_side(left_latex, right_latex):
    return r"\begin{tabular}{ccccccc}\hline"+left_latex+r"\\ \hline\end{tabular}\quad\begin{tabular}{ccccccc}\hline"+right_latex+r"\\ \hline\end{tabular}"

def fix_latex(input_string):
    # Replace '%' with '\%'
    replacements = {"%":"\\%",
                    "pRef_method":"Met.",
                    "{SA}": r"{\rotcell{SA}}", # note that SA is a subset of SAT\_50 etc.., so it causes some issues
                    "SAT_S": "SAT\_20",
                    "SAT_M": "SAT\_50",
                    "SAT_L": "SAT\_100",
                    "GC_L": "GC\_anna",
                    "GC_S": "GC\_jean",
                    "uniform": "RS",
                    "kind": "tree",
                    r"\multirow[c]{12}" : r"\hline \multirow[c]{12}",
                    r"& \multirow[c]{3}" : r"\cline{2-7} & \multirow[c]{3}",
                    r"} \cline{2-7}" : "} ",
                   "\\font-weightbold": "",
                    "≪": "\ll "}

    texts_to_rotate = ["problem", "BT", "GC\_anna", "GC\_jean", "SAT\_20",  "SAT\_50",  "SAT\_100", "Met.", "GA", "Tabu", "RS", "depth"]

    for item_to_rotate in texts_to_rotate:
        replacements[item_to_rotate] = r"\rotcell{"+item_to_rotate+"}"

    modified_string = str(input_string)
    for orig, replacement in replacements.items():
        modified_string = modified_string.replace(orig, replacement)
    
    return modified_string


def pivot_table_as_latex(pivot_table):
    latex_text = pivot_table.to_latex(convert_css=True)
    latex_text = fix_latex(latex_text)
    return latex_text

In [7]:
pRef_size = 10000
depths = [3, 4, 5]


usable_data = accuracy_data.copy()
usable_data = usable_data[usable_data["pRef_size"] == pRef_size] 
usable_data = usable_data[usable_data["depth"].isin(depths)]

independent_variables = ["problem", "pRef_method", "kind", "depth"]
dependent_variables = ["r_sq"]


problems = ["BT", "GC_S", "GC_L", "SAT_S", "SAT_M", "SAT_L"]

left_problems, right_problems = problems[:3], problems[3:]

def make_table_for_problems(problem_subset):
    with_right_problems = usable_data[usable_data['problem'].isin(problem_subset)]
    pivot_table = with_right_problems.pivot_table(index = ["problem", "pRef_method", "depth"], 
                                        columns = ["kind"], 
                                        values = dependent_variables[0])
    return style_pivot_table(pivot_table)



left_table = make_table_for_problems(left_problems)
right_table = make_table_for_problems(right_problems)

display(left_table)
display(right_table)

left_table_latex = pivot_table_as_latex(left_table)
right_table_latex = pivot_table_as_latex(right_table)

full_table_latex = put_latex_tables_side_by_side(left_table_latex, right_table_latex)

print("left table:")
print(left_table_latex)

print("\n\n\n\n\n\nright table")
print(right_table_latex)



# pivot_table = usable_data.pivot_table(index = ["problem", "pRef_method", "depth"], 
#                                         columns = ["kind"], 
#                                         values = dependent_variables[0])





# for problem in usable_data['problem'].unique():
#     with_right_problem = usable_data[usable_data['problem'] == problem]
#     pivot_table = with_right_problem.pivot_table(index = ["pRef_method", "depth"], 
#                                         columns = ["kind"], 
#                                         values = dependent_variables[0])

#     pivot_table = style_pivot_table(pivot_table)
#     print(fix_latex(f"{problem = }"))
#     print_pivot_table_as_latex(pivot_table)
    
    # Display the styled dataframe
    #display(styled_df)
    
    #display(pivot_table)

left table:
\begin{tabular}{lllllll}
 &  & tree & PS-SW & PS-SWA & IAI & Trad. \\
\rotcell{problem} & \rotcell{Met.} & \rotcell{depth} &  &  &  &  \\
\hline \multirow[c]{12}{*}{\rotcell{BT}}  & \multirow[c]{3}{*}{\rotcell{GA}} & 3 & \bfseries 85.1\% & 81.3\% & 66.6\% & 77.3\% \\
 &  & 4 & \bfseries 89.1\% & 86.2\% & 78.0\% & 83.4\% \\
 &  & 5 & \bfseries 91.2\% & 89.1\% & 83.8\% & 86.9\% \\
 \cline{2-7} & \multirow[c]{3}{*}{\rotcell{SA}} & 3 & 78.6\% & 72.0\% & 65.0\% & \bfseries 79.2\% \\
 &  & 4 & 84.2\% & 80.3\% & 82.1\% & \bfseries 88.0\% \\
 &  & 5 & 88.1\% & 86.1\% & 89.8\% & \bfseries 92.8\% \\
 \cline{2-7} & \multirow[c]{3}{*}{\rotcell{Tabu}} & 3 & 26.0\% & 24.1\% & 21.9\% & \bfseries 30.3\% \\
 &  & 4 & 34.3\% & 32.7\% & 32.2\% & \bfseries 39.3\% \\
 &  & 5 & 42.5\% & 41.0\% & 41.8\% & \bfseries 48.8\% \\
 \cline{2-7} & \multirow[c]{3}{*}{\rotcell{RS}} & 3 & 3.6\% & 5.2\% & 11.2\% & \bfseries 14.5\% \\
 &  & 4 & 4.9\% & 6.1\% & 14.3\% & \bfseries 17.2\% \\
 &  & 5 & 6.1\% & 6.

In [8]:

pRef_size = 10000
depths = [3, 4, 5]


problems = ["BT", "GC_S", "GC_L", "SAT_S", "SAT_M", "SAT_L"]

left_problems, middle_problems, right_problems = problems[:2], problems[2:4], problems[4:]

def make_table_for_problems(problem_subset):
    with_right_problems = statistical_data[statistical_data['problem'].isin(problem_subset)]
    threshold = 0.005
    # Add the new 'successfull' column with "<<0.05" or the original p-value
    with_right_problems['p-value*'] = with_right_problems['p_value_sw'].apply(lambda x: r'$\ll \alpha$' if x < threshold else str(round(x, 2))[:4])
    
    display(with_right_problems)

    pivot_table = with_right_problems.pivot_table(index = ["problem", "metaheuristic"], 
                                                  columns = ["depth"],
                                        values = ["p-value*"],
                                                  aggfunc=lambda x: x)
    
    
    return pivot_table



table = make_table_for_problems(problems)

display(table)

latex_text = table.to_latex()
latex_text = fix_latex(latex_text)
print(latex_text)

#left_table_latex = pivot_table_as_latex(left_table)
#right_table_latex = pivot_table_as_latex(right_table)

#full_table_latex = put_latex_tables_side_by_side(left_table_latex, right_table_latex)

# print("left table:")
# print(left_table_latex)
# 
# print("\n\n\n\n\n\nright table")
# print(right_table_latex)


,problem,depth,metaheuristic,p_value_sw,p_value_swa,winning_competitor,p-value*
0,SAT_S,3,uniform,1.000000e+00,1.000000e+00,Trad.,1.0
1,SAT_S,3,GA,2.440937e-03,1.000000e+00,Trad.,$\ll \alpha$
2,SAT_S,3,SA,9.999974e-01,1.000000e+00,Trad.,1.0
3,SAT_S,3,Tabu,1.001752e-04,1.000000e+00,Trad.,$\ll \alpha$
4,SAT_S,4,uniform,1.000000e+00,1.000000e+00,Trad.,1.0
...,...,...,...,...,...,...,...
67,BT,4,Tabu,9.963082e-01,1.000000e+00,Trad.,1.0
68,BT,5,uniform,1.000000e+00,1.000000e+00,Trad.,1.0
69,BT,5,GA,4.464741e-31,2.666489e-11,Trad.,$\ll \alpha$
70,BT,5,SA,1.000000e+00,1.000000e+00,Trad.,1.0


p-value*                            
depth                             3             4             5
problem metaheuristic                                          
BT      GA             $\ll \alpha$  $\ll \alpha$  $\ll \alpha$
        SA                     0.48           1.0           1.0
        Tabu                    1.0           1.0           1.0
        uniform                 1.0           1.0           1.0
GC_L    GA             $\ll \alpha$  $\ll \alpha$  $\ll \alpha$
        SA                     0.01           1.0           1.0
        Tabu                   0.02          0.93           1.0
        uniform                 1.0           1.0           1.0
GC_S    GA             $\ll \alpha$  $\ll \alpha$  $\ll \alpha$
        SA                     0.95           1.0           1.0
        Tabu                   0.99           1.0           1.0
        uniform                 1.0           1.0           1.0
SAT_L   GA             $\ll \alpha$  $\ll \alpha$  $\ll \alpha$
        SA             $\ll \alpha$  $\ll \alpha$          0.01
        Tabu           $\ll \alpha$  $\ll \alpha$  $\ll \alpha$
        uniform                 1.0           1.0           1.0
SAT_M   GA             $\ll \alpha$  $\ll \alpha$  $\ll \alpha$
        SA             $\ll \alpha$  $\ll \alpha$  $\ll \alpha$
        Tabu           $\ll \alpha$  $\ll \alpha$  $\ll \alpha$
        uniform                 1.0           1.0           1.0
SAT_S   GA             $\ll \alpha$  $\ll \alpha$          0.02
        SA                      1.0           1.0           1.0
        Tabu           $\ll \alpha$          0.01  $\ll \alpha$
        uniform                 1.0           1.0           1.0

\begin{tabular}{lllll}
\toprule
 &  & \multicolumn{3}{r}{p-value*} \\
 & \rotcell{depth} & 3 & 4 & 5 \\
\rotcell{problem} & metaheuristic &  &  &  \\
\midrule
\multirow[t]{4}{*}{\rotcell{BT}} & \rotcell{GA} & $\ll \alpha$ & $\ll \alpha$ & $\ll \alpha$ \\
 & SA & 0.48 & 1.0 & 1.0 \\
 & \rotcell{Tabu} & 1.0 & 1.0 & 1.0 \\
 & \rotcell{RS} & 1.0 & 1.0 & 1.0 \\
\cline{1-5}
\multirow[t]{4}{*}{\rotcell{GC\_anna}} & \rotcell{GA} & $\ll \alpha$ & $\ll \alpha$ & $\ll \alpha$ \\
 & SA & 0.01 & 1.0 & 1.0 \\
 & \rotcell{Tabu} & 0.02 & 0.93 & 1.0 \\
 & \rotcell{RS} & 1.0 & 1.0 & 1.0 \\
\cline{1-5}
\multirow[t]{4}{*}{\rotcell{GC\_jean}} & \rotcell{GA} & $\ll \alpha$ & $\ll \alpha$ & $\ll \alpha$ \\
 & SA & 0.95 & 1.0 & 1.0 \\
 & \rotcell{Tabu} & 0.99 & 1.0 & 1.0 \\
 & \rotcell{RS} & 1.0 & 1.0 & 1.0 \\
\cline{1-5}
\multirow[t]{4}{*}{\rotcell{SAT\_100}} & \rotcell{GA} & $\ll \alpha$ & $\ll \alpha$ & $\ll \alpha$ \\
 & SA & $\ll \alpha$ & $\ll \alpha$ & 0.01 \\
 & \rotcell{Tabu} & $\ll \alpha$ & $\ll \a

In [9]:
result = []
print(accuracy_data["problem"].unique())
print(accuracy_data["pRef_method"].unique())
for method in accuracy_data["pRef_method"].unique():
    for problem in accuracy_data["problem"].unique():
        for depth in [3, 4, 5]:
            appropriate_data = filter_dataframe(accuracy_data, depth = depth, pRef_method = method, problem = problem, pRef_size=10000)
            trad_data = appropriate_data[appropriate_data["kind"] == "Trad."]["r_sq"][:100]
            own_data = appropriate_data[appropriate_data["kind"] == "PS-SW"]["r_sq"][:100]
            p_value = mannwhitneyu(x = own_data, y = trad_data, alternative="greater").pvalue
            diff = np.average(own_data)-np.average(trad_data)
            
            p_value_string = "0.001" if p_value < 0.001 else round(p_value, 3)
            res = f"{round(diff*100, 2)}%, {p_value_string}"
            if p_value < 0.05:
                res = "\\bfseries "+res
            
            
            result.append({"method":method,
                           "problem":problem,
                           "depth":depth,
                           "res":res})
            
        
improvements = pd.DataFrame(result)
display(improvements)
pivot_table = improvements.pivot_table(index=["problem", "method"],
                                       columns="depth",
                                       values=["res"],
                                       aggfunc=lambda x:x)

#pivot_table = pivot_table.mul(100).round(1).astype(str) + "%"    

display(pivot_table)

latex_code = pivot_table.to_latex()

def fix_latex(input_string):
    # Replace '%' with '\%'
    replacements = {"%":"\\%",
                    "pRef_method":"Met.",
                    "BT":"Staff R.",
                    "SAT_S": "SAT\_20",
                    "SAT_M": "SAT\_50",
                    "SAT_L": "SAT\_100",
                    "GC_L": "GC\_anna",
                    "GC_S": "GC\_jean",
                    "uniform": "RS",
                    "kind": "tree",
                   "\\font-weightbold": "",
                    "res": "average R^2 improvemennt between PS-SW and Trad, with p-value",
                    "≪": "\ll "}


    modified_string = str(input_string)
    for orig, replacement in replacements.items():
        modified_string = modified_string.replace(orig, replacement)
    
    return modified_string
latex_code = fix_latex(latex_code)
print(latex_code)


['SAT_S' 'SAT_M' 'SAT_L' 'GC_S' 'GC_L' 'BT']
['uniform' 'GA' 'SA' 'Tabu']


,method,problem,depth,res
0,uniform,SAT_S,3,"-19.28%, 1.0"
1,uniform,SAT_S,4,"-26.19%, 1.0"
2,uniform,SAT_S,5,"-31.55%, 1.0"
3,uniform,SAT_M,3,"-1.57%, 1.0"
4,uniform,SAT_M,4,"-2.56%, 1.0"
...,...,...,...,...
67,Tabu,GC_L,4,"-0.54%, 0.898"
68,Tabu,GC_L,5,"-3.9%, 1.0"
69,Tabu,BT,3,"-4.01%, 0.994"
70,Tabu,BT,4,"-4.69%, 0.989"


res                           \
depth                                  3                        4   
problem method                                                      
BT      GA         \bfseries 7.9%, 0.001   \bfseries 5.85%, 0.001   
        SA                  -0.5%, 0.438              -3.72%, 1.0   
        Tabu               -4.01%, 0.994            -4.69%, 0.989   
        uniform             -10.91%, 1.0             -12.34%, 1.0   
GC_L    GA        \bfseries 8.43%, 0.001   \bfseries 6.83%, 0.001   
        SA        \bfseries 2.03%, 0.012              -3.48%, 1.0   
        Tabu      \bfseries 0.83%, 0.013            -0.54%, 0.898   
        uniform              -0.53%, 1.0              -0.99%, 1.0   
GC_S    GA        \bfseries 8.34%, 0.001   \bfseries 6.07%, 0.001   
        SA                 -2.44%, 0.964              -6.06%, 1.0   
        Tabu               -1.56%, 0.991              -5.06%, 1.0   
        uniform              -0.45%, 1.0              -0.55%, 1.0   
SAT_L   GA       \bfseries 15.92%, 0.001  \bfseries 13.11%, 0.001   
        SA       \bfseries 13.22%, 0.001   \bfseries 7.02%, 0.001   
        Tabu      \bfseries 5.94%, 0.001   \bfseries 7.27%, 0.001   
        uniform             -0.8%, 0.996            -0.87%, 0.998   
SAT_M   GA       \bfseries 16.69%, 0.001  \bfseries 15.37%, 0.001   
        SA       \bfseries 11.74%, 0.001   \bfseries 9.04%, 0.001   
        Tabu       \bfseries 4.4%, 0.001   \bfseries 5.31%, 0.001   
        uniform              -1.57%, 1.0              -2.56%, 1.0   
SAT_S   GA        \bfseries 1.11%, 0.001   \bfseries 0.52%, 0.003   
        SA                   -7.01%, 1.0              -9.23%, 1.0   
        Tabu     \bfseries -0.41%, 0.001   \bfseries -1.4%, 0.014   
        uniform             -19.28%, 1.0             -26.19%, 1.0   

                                          
depth                                  5  
problem method                            
BT      GA        \bfseries 4.45%, 0.001  
        SA                    -4.7%, 1.0  
        Tabu               -6.01%, 0.998  
        uniform             -12.86%, 1.0  
GC_L    GA         \bfseries 5.5%, 0.001  
        SA                   -7.91%, 1.0  
        Tabu                  -3.9%, 1.0  
        uniform               -1.1%, 1.0  
GC_S    GA        \bfseries 4.49%, 0.001  
        SA                   -7.07%, 1.0  
        Tabu                 -8.98%, 1.0  
        uniform            -0.18%, 0.819  
SAT_L   GA       \bfseries 10.79%, 0.001  
        SA        \bfseries 1.16%, 0.003  
        Tabu      \bfseries 7.04%, 0.001  
        uniform              -1.31%, 1.0  
SAT_M   GA       \bfseries 13.32%, 0.001  
        SA        \bfseries 5.26%, 0.001  
        Tabu      \bfseries 6.21%, 0.001  
        uniform              -3.66%, 1.0  
SAT_S   GA       \bfseries -0.34%, 0.017  
        SA                  -11.97%, 1.0  
        Tabu     \bfseries -1.97%, 0.007  
        uniform             -31.55%, 1.0

\begin{tabular}{lllll}
\toprule
 &  & \multicolumn{3}{r}{average R^2 improvemennt between PS-SW and Trad, with p-value} \\
 & depth & 3 & 4 & 5 \\
problem & method &  &  &  \\
\midrule
\multirow[t]{4}{*}{Staff R.} & GA & \bfseries 7.9\%, 0.001 & \bfseries 5.85\%, 0.001 & \bfseries 4.45\%, 0.001 \\
 & SA & -0.5\%, 0.438 & -3.72\%, 1.0 & -4.7\%, 1.0 \\
 & Tabu & -4.01\%, 0.994 & -4.69\%, 0.989 & -6.01\%, 0.998 \\
 & RS & -10.91\%, 1.0 & -12.34\%, 1.0 & -12.86\%, 1.0 \\
\cline{1-5}
\multirow[t]{4}{*}{GC\_anna} & GA & \bfseries 8.43\%, 0.001 & \bfseries 6.83\%, 0.001 & \bfseries 5.5\%, 0.001 \\
 & SA & \bfseries 2.03\%, 0.012 & -3.48\%, 1.0 & -7.91\%, 1.0 \\
 & Tabu & \bfseries 0.83\%, 0.013 & -0.54\%, 0.898 & -3.9\%, 1.0 \\
 & RS & -0.53\%, 1.0 & -0.99\%, 1.0 & -1.1\%, 1.0 \\
\cline{1-5}
\multirow[t]{4}{*}{GC\_jean} & GA & \bfseries 8.34\%, 0.001 & \bfseries 6.07\%, 0.001 & \bfseries 4.49\%, 0.001 \\
 & SA & -2.44\%, 0.964 & -6.06\%, 1.0 & -7.07\%, 1.0 \\
 & Tabu & -1.56\%, 0.991 & -5.06\